In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import os
import requests
import io
import zipfile

/tmp/ipykernel_1915/2038222792.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
dfs : dict[str, pd.DataFrame] = { file.split('.')[0] : pd.read_csv(f'../data/core/mb-alloc/{file}', dtype=str) for file in os.listdir('../data/core/mb-alloc')}

In [4]:
dfs = dict(sorted(dfs.items()))

In [5]:
dfs['MB_2021_AUST']['MB_CODE_2021'].is_unique, dfs['SAL_2021_AUST']['MB_CODE_2021'].is_unique, dfs['POA_2021_AUST']['MB_CODE_2021'].is_unique, dfs['SUA_2021_AUST']['SA2_CODE_2021'].is_unique

(True, True, True, True)

In [6]:
dfs['MB_2021_AUST']['SA2_CODE_2021'].unique().size, dfs['SUA_2021_AUST']['SA2_CODE_2021'].unique().size

(2473, 2473)

In [7]:
dfs['MB_2021_AUST']['MB_CODE_2021'].unique().size, dfs['POA_2021_AUST']['MB_CODE_2021'].unique().size

(368286, 368286)

In [8]:
dfs['MB_2021_AUST']['MB_CODE_2021'].unique().size, dfs['POA_2021_AUST']['MB_CODE_2021'].unique().size

(368286, 368286)

In [9]:
dfMB = pd.merge(dfs['MB_2021_AUST'], dfs['SAL_2021_AUST'], on='MB_CODE_2021', how='left', suffixes=('', '_SAL_'))
dfMB = pd.merge(dfMB, dfs['POA_2021_AUST'], on='MB_CODE_2021', how='left', suffixes=('', '_POA_'))
dfMB = pd.merge(dfMB, dfs['LGA_2023_AUST'], on='MB_CODE_2021', how='left', suffixes=('', '_LGA_'))
dfMB = pd.merge(dfMB, dfs['SUA_2021_AUST'], on='SA2_CODE_2021', how='left', suffixes=('', '_SUA_'))
dfMB.drop(columns=[c for c in dfMB.columns if c.endswith('_')], inplace=True)

In [10]:
dfMB['AREA_ALBERS_SQKM'] = dfMB['AREA_ALBERS_SQKM'].astype(float)

In [59]:
dfMB[dfMB['SUA_CODE_2021'].isna()]

,MB_CODE_2021,MB_CATEGORY_2021,CHANGE_FLAG_2021,CHANGE_LABEL_2021,SA1_CODE_2021,SA2_CODE_2021,SA2_NAME_2021,SA3_CODE_2021,SA3_NAME_2021,SA4_CODE_2021,...,AREA_ALBERS_SQKM,ASGS_LOCI_URI_2021,SAL_CODE_2021,SAL_NAME_2021,POA_CODE_2021,POA_NAME_2021,LGA_CODE_2023,LGA_NAME_2023,SUA_CODE_2021,SUA_NAME_2021


In [78]:
df_subs = dfMB.groupby(['SAL_CODE_2021', 'POA_CODE_2021']).aggregate({
    'SAL_NAME_2021' : 'first',
    'POA_NAME_2021' : 'first',
    'GCCSA_CODE_2021' : 'unique',
    'GCCSA_NAME_2021' : 'unique',
    'LGA_CODE_2023' : 'unique',
    'LGA_NAME_2023' : 'unique',
    'SUA_CODE_2021' : 'unique',
    'SUA_NAME_2021' : 'unique',
    'STATE_CODE_2021' : 'first',
    'STATE_NAME_2021' : 'first',
    'AUS_CODE_2021' : 'first',
    'AUS_NAME_2021' : 'first',
    'AREA_ALBERS_SQKM' : 'sum',
    'MB_CATEGORY_2021' : 'unique',
})

df_subs.reset_index(inplace=True)

df_subs = df_subs[['SAL_CODE_2021','POA_CODE_2021', 'STATE_CODE_2021']].rename(columns={
    'SAL_CODE_2021': 'suburb_id',
    'POA_CODE_2021': 'postcode_id',
    'STATE_CODE_2021': 'state_idx'
})

In [11]:
df_suburbs = dfMB.groupby(['SAL_CODE_2021', 'POA_CODE_2021']).aggregate({
    'SAL_NAME_2021' : 'first',
    'POA_NAME_2021' : 'first',
    'GCCSA_CODE_2021' : 'unique',
    'GCCSA_NAME_2021' : 'unique',
    'LGA_CODE_2023' : 'unique',
    'LGA_NAME_2023' : 'unique',
    'SUA_CODE_2021' : 'unique',
    'SUA_NAME_2021' : 'unique',
    'STATE_CODE_2021' : 'first',
    'STATE_NAME_2021' : 'first',
    'AUS_CODE_2021' : 'first',
    'AUS_NAME_2021' : 'first',
    'AREA_ALBERS_SQKM' : 'sum',
    'MB_CATEGORY_2021' : 'unique',
}).reset_index().rename(columns={
    'SAL_CODE_2021' : 'suburb_id',
    'SAL_NAME_2021' : 'suburb_name',
    'POA_CODE_2021' : 'postcode_id',
    'POA_NAME_2021' : 'postcode',
    'GCCSA_CODE_2021' : 'gccsa_id',
    'GCCSA_NAME_2021' : 'capital_area',
    'LGA_CODE_2023' : 'lga_id',
    'LGA_NAME_2023' : 'lga_name',
    'SUA_CODE_2021' : 'sua_id',
    'SUA_NAME_2021' : 'urban_area',
    'STATE_CODE_2021' : 'state_id',
    'STATE_NAME_2021' : 'state_name',
    'AUS_CODE_2021' : 'country_id',
    'AUS_NAME_2021' : 'country_name',
    'AREA_ALBERS_SQKM' : 'area_sqkm',
    'MB_CATEGORY_2021' : 'categories',
})


In [12]:
sua_ids = dfs['SUA_2021_AUST']['SUA_CODE_2021'].unique()

In [13]:
dfk_sua_suburbs = {}
for sua_id in sua_ids:
    dfk_sua_suburbs[sua_id] = df_suburbs[df_suburbs['sua_id'].apply(lambda x: sua_id in x)]

In [80]:
df_subs

,suburb_id,postcode_id,state_idx
0,10001,2850,1
1,10002,2176,1
2,10003,2046,1
3,10004,2795,1
4,10005,2795,1
...,...,...,...
15535,90004,2899,9
15536,90005,6799,9
15537,99494,9494,9
15538,99797,9797,9


In [20]:
gdf_suburbs : gpd.GeoDataFrame = gpd.GeoDataFrame(pd.concat([gpd.read_file(f'../data/core/suburbs/{file}') for file in os.listdir('../data/core/suburbs')], ignore_index=True))

In [64]:
len(df_subs), len(gdf_suburbs)

(15540, 15515)

In [72]:
df_subs.reset_index(inplace=True)

In [92]:
gdfx = pd.merge(gdf_suburbs, df_subs, how='outer')

In [91]:
dfs['POA_2021_AUST'][dfs['POA_2021_AUST']['POA_CODE_2021'] == '0800']

,MB_CODE_2021,POA_CODE_2021,POA_NAME_2021,AUS_CODE_2021,AUS_NAME_2021,AREA_ALBERS_SQKM,ASGS_LOCI_URI_2021


In [87]:
df_subs[df_subs['postcode_id'] == '0810']

,suburb_id,postcode_id,state_idx


In [93]:
gdfx[gdfx['state_id'].isna()]

,suburb_id,postcode_id,area_sqkm,state_id,country_id,suburb,postcode,state_name,country_name,state,geometry,state_idx
790,10778,4385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1
3873,13824,4385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1
4597,19494,9494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1
4598,19797,9797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1
5857,21235,3996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,2
...,...,...,...,...,...,...,...,...,...,...,...,...
15706,70302,872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,7
15708,70303,832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,7
15861,99494,9494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,9
15862,99797,9797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,9


In [85]:
gdfx[gdfx['state_idx'].isna()]

,suburb_id,postcode_id,area_sqkm,state_id,country_id,suburb,postcode,state_name,country_name,state,geometry,state_idx
2550,70073,0800,3.1731,7,AUS,Darwin City,0800,Northern Territory,Australia,NT,"POLYGON ((130.83878 -12.46523, 130.83874 -12.4...",NaN
2551,70162,0810,4.3642,7,AUS,Lee Point,0810,Northern Territory,Australia,NT,"POLYGON ((130.88270 -12.35092, 130.87972 -12.3...",NaN
2552,70192,0810,1.0424,7,AUS,Moil,0810,Northern Territory,Australia,NT,"POLYGON ((130.87829 -12.39437, 130.87829 -12.3...",NaN
2553,70198,0810,1.7106,7,AUS,Muirhead,0810,Northern Territory,Australia,NT,"POLYGON ((130.89167 -12.36210, 130.89164 -12.3...",NaN
2554,70201,0810,0.8426,7,AUS,Nakara,0810,Northern Territory,Australia,NT,"POLYGON ((130.87451 -12.37424, 130.87371 -12.3...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
14378,50894,0872,4.8335,5,AUS,Mantamaru,0872,Western Australia,Australia,WA,"POLYGON ((127.66076 -25.86638, 127.66003 -25.8...",NaN
14379,51212,0872,10.3261,5,AUS,Patjarr,0872,Western Australia,Australia,WA,"POLYGON ((126.33468 -24.62853, 126.29858 -24.6...",NaN
14380,51528,0872,2.0367,5,AUS,Warakurna,0872,Western Australia,Australia,WA,"POLYGON ((128.27698 -24.99628, 128.27673 -24.9...",NaN
14381,51200,0872,4.3599,5,AUS,Papulankutja,0872,Western Australia,Australia,WA,"POLYGON ((128.28980 -25.98239, 128.29074 -25.9...",NaN


In [21]:
gdf_suburbs.columns

Index(['suburb_id', 'postcode_id', 'area_sqkm', 'state_id', 'country_id',
       'suburb', 'postcode', 'state_name', 'country_name', 'state',
       'geometry'],
      dtype='object')

In [23]:
df_suburbs[['suburb_id', 'postcode_id', 'sua_id']].head()

,suburb_id,postcode_id,sua_id
0,10001,2850,[1000]
1,10002,2176,[1031]
2,10003,2046,[1031]
3,10004,2795,[1005]
4,10005,2795,[1000]


In [39]:
len(df_suburbs), len(gdf_suburbs)

(15540, 15515)

In [46]:
gdf_suburbs.apply(lambda x: (x['suburb_id'], x['postcode_id']), axis=1).is_unique

True

In [47]:
df_suburbs.apply(lambda x: (x['suburb_id'], x['postcode_id']), axis=1).is_unique

True

In [48]:
gdf_sua = pd.merge(gdf_suburbs, df_suburbs[['suburb_id', 'postcode_id', 'sua_id']], on=['suburb_id', 'postcode_id'], how='left')

In [53]:
gdf_sua_2 = pd.merge(gdf_suburbs, df_suburbs[['suburb_id', 'postcode_id', 'sua_id']], on=['suburb_id', 'postcode_id'], how='outer')

In [56]:
gdf_sua_2[gdf_sua_2['sua_id'].isna()]['postcode'].unique()

array(['0872', '0822', '0846', '0810', '0870', '0885', '0873', '0812',
       '0830', '0852', '0832', '0845', '0820', '0828', '0838', '0854',
       '0862', '0839', '0850', '0800', '0841', '0840', '0875', '0880',
       '0836', '0829', '0835', '0837', '0874', '0886', '0847', '0860',
       '0853', '0834'], dtype=object)

In [27]:
gdf_sua = gpd.GeoDataFrame(gdf_sua)

In [28]:
os.makedirs('../data/sua-suburbs', exist_ok=True)

In [51]:
gdf_sua[gdf_sua['sua_id'].apply(lambda x: (not isinstance(x, np.ndarray)))]

,suburb_id,postcode_id,area_sqkm,state_id,country_id,suburb,postcode,state_name,country_name,state,geometry,sua_id
2550,70073,0800,3.1731,7,AUS,Darwin City,0800,Northern Territory,Australia,NT,"POLYGON ((130.83878 -12.46523, 130.83874 -12.4...",NaN
2551,70162,0810,4.3642,7,AUS,Lee Point,0810,Northern Territory,Australia,NT,"POLYGON ((130.88270 -12.35092, 130.87972 -12.3...",NaN
2552,70192,0810,1.0424,7,AUS,Moil,0810,Northern Territory,Australia,NT,"POLYGON ((130.87829 -12.39437, 130.87829 -12.3...",NaN
2553,70198,0810,1.7106,7,AUS,Muirhead,0810,Northern Territory,Australia,NT,"POLYGON ((130.89167 -12.36210, 130.89164 -12.3...",NaN
2554,70201,0810,0.8426,7,AUS,Nakara,0810,Northern Territory,Australia,NT,"POLYGON ((130.87451 -12.37424, 130.87371 -12.3...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
14378,50894,0872,4.8335,5,AUS,Mantamaru,0872,Western Australia,Australia,WA,"POLYGON ((127.66076 -25.86638, 127.66003 -25.8...",NaN
14379,51212,0872,10.3261,5,AUS,Patjarr,0872,Western Australia,Australia,WA,"POLYGON ((126.33468 -24.62853, 126.29858 -24.6...",NaN
14380,51528,0872,2.0367,5,AUS,Warakurna,0872,Western Australia,Australia,WA,"POLYGON ((128.27698 -24.99628, 128.27673 -24.9...",NaN
14381,51200,0872,4.3599,5,AUS,Papulankutja,0872,Western Australia,Australia,WA,"POLYGON ((128.28980 -25.98239, 128.29074 -25.9...",NaN


In [29]:
for sua_id in sua_ids:
    gdf_sua[gdf_sua['sua_id'].apply(lambda x: sua_id in x)].drop(columns=['sua_id']).to_file('../data/sua-suburbs/sua-suburbs-{sua_id}.geojson', driver='GeoJSON')

TypeError: argument of type 'float' is not iterable

In [ ]:


gdf_suburbs.to_file('data/custom/suburbs-greater-sydney.geojson', driver='GeoJSON')


